In [1]:
# import libraries

import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.17.0


In [8]:
import tensorflow as tf
import os
import numpy as np
from keras.datasets import imdb
from keras.preprocessing import sequence
from tensorflow import keras

In [2]:
# get data files
train_file_path = "https://raw.githubusercontent.com/valemicolgarcia/TensorFlow/main/SMS%20Text%20Classifier/train-data.tsv"
test_file_path = "https://raw.githubusercontent.com/valemicolgarcia/TensorFlow/main/SMS%20Text%20Classifier/valid-data.tsv"

In [5]:
train = pd.read_csv(train_file_path, sep='\t')
train.head()

,ham,"ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it."
0,ham,you can never do nothing
1,ham,"now u sound like manky scouse boy steve,like! ..."
2,ham,mum say we wan to go then go... then she can s...
3,ham,never y lei... i v lazy... got wat? dat day ü ...
4,ham,in xam hall boy asked girl tell me the startin...


In [6]:
test = pd.read_csv(test_file_path, sep='\t')
test.head()

,ham,i am in hospital da. . i will return home in evening
0,ham,"not much, just some textin'. how bout you?"
1,ham,i probably won't eat at all today. i think i'm...
2,ham,don‘t give a flying monkeys wot they think and...
3,ham,who are you seeing?
4,ham,your opinion about me? 1. over 2. jada 3. kusr...


In [13]:
train_label = train['ham']
train_label.head()

0    ham
1    ham
2    ham
3    ham
4    ham
Name: ham, dtype: object

In [14]:
train_x = train.drop(columns='ham')
train_x.head()

,"ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it."
0,you can never do nothing
1,"now u sound like manky scouse boy steve,like! ..."
2,mum say we wan to go then go... then she can s...
3,never y lei... i v lazy... got wat? dat day ü ...
4,in xam hall boy asked girl tell me the startin...


In [15]:
test_label = test['ham']
test_x = test.drop(columns='ham')

In [16]:
test_label.head()

0    ham
1    ham
2    ham
3    ham
4    ham
Name: ham, dtype: object

In [17]:
test_x.head()

,i am in hospital da. . i will return home in evening
0,"not much, just some textin'. how bout you?"
1,i probably won't eat at all today. i think i'm...
2,don‘t give a flying monkeys wot they think and...
3,who are you seeing?
4,your opinion about me? 1. over 2. jada 3. kusr...


In [30]:
print(train_x.isnull().sum())

ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.    0
dtype: int64


In [32]:
print(test_x.isnull().sum())

i am in hospital da. . i will return home in evening    0
dtype: int64


In [33]:
# Codificar etiquetas
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_label = label_encoder.fit_transform(train_label)
test_label = label_encoder.fit_transform(test_label)

In [51]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preprocesamiento y tokenización
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_x)  # Ajusta el tokenizer a tus datos de entrenamiento

# Tokeniza los textos
X_sequences = tokenizer.texts_to_sequences(train_x)
X_sequences_test = tokenizer.texts_to_sequences(test_x)

# Aplica padding a las secuencias
X_padded = pad_sequences(X_sequences, maxlen=100)
X_padded_test = pad_sequences(X_sequences_test, maxlen=100)


----------------------------------------------------------------

In [52]:
len(tokenizer.word_index) + 1

29

In [11]:
#train_data = sequence.pad_sequences(train, MAXLEN)
#test_data = sequence.pad_sequences(test, MAXLEN)

In [53]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index) + 1, 32), #transforma palabras en vectores de numeros reales que capturan relacioones semanticas entre palabras
    tf.keras.layers.LSTM(32), #capa recurrente capaz de aprender dependencias a largo plazo de secuencias de texto
    tf.keras.layers.Dense(1, activation='sigmoid') #capa densa (totalmente conectada), la funcion de activacion sigmoide comprime la salida en el rango [0,1] lo cual es ideal para tareas de clasificacion binaria
    #el modelo luego puede decidir si una instancia pertenece a una clase especifica comparando esta probabilidad con un umbral, tipicamente 0.5
])

In [54]:
model.compile(loss='binary_crossentropy', 
              optimizer='rmsprop', 
              metrics=['acc'])


In [55]:
train_label.shape

(4178,)

In [56]:
history = model.fit (X_padded, train_label, epochs=10)

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 1
'y' sizes: 4178


-----------------------------------------------

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):



  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
